<a href="https://colab.research.google.com/github/zhenghaojiang/rl_dsge/blob/main/SMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import numpy.random as rnd
import numpy.linalg as lin
from sympy import *
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import pandas as pd
from tqdm import tqdm

from joblib import Parallel, delayed

ModuleNotFoundError: ignored

In [3]:
# Simulation method --- linear
r = 1/0.96 - 1
delta = 0.15
def linear_sim(params,steps=1000,foresteps=200,times=1):
  theta, rho, sigma = params

  k_st = (((r+delta)*(1+0.01*delta)-0.005*(delta**2))/theta) ** (1/(theta-1))
  I_st = delta*k_st
  m1 = theta*(theta-1)*(k_st**(theta-2))-0.01*(I_st**2)/(k_st**3)-0.01*(1-delta)*I_st/(k_st**2)
  m2 = 0.01*(I_st/(k_st**2)+(1-delta)/k_st)

  A = Symbol('A')
  B = Symbol('B')
  solved_value = solve([0.01*(1+r)*(A/k_st-I_st/(k_st**2))-(1-delta+A)*(m1+m2*A), 
              0.01*(1+r)*B/k_st-theta*(k_st**(theta-1))*rho-m1*B-m2*B*(A+rho)], 
              [A, B])
  
  kab = np.array(solved_value)[0]
  z0 = 1
  k0 = 1
  def action(k, z):
    I_tilt = kab[0]*(k-k_st)+kab[1]*(z-1)
    return I_tilt+I_st

  I0 = action(k0,z0)
  def ztrans(z=z0):
    lnz = np.log(z)
    lnz_new = rho*lnz + np.random.normal(0, sigma, 1)[0]
    return np.exp(lnz_new)
  def ktrans(k=k0, I=I0):
    k_new = (1-delta)*k+I
    return k_new
  
  for t in range(times):
    k_sim = []
    I_sim = []
    z_sim = []
    for i in range(steps+foresteps):
      if i==0:
        z_sim.append(z0)
        k_sim.append(k0)
        I_sim.append(I0)
      else:
        z_sim.append(ztrans(z_sim[i-1]))
        k_sim.append(ktrans(k_sim[i-1],I_sim[i-1]))
        I_sim.append(action(k_sim[i],z_sim[i]))
    if t==0:
      k_sim_all = np.array([k_sim[foresteps:]])
      I_sim_all = np.array([I_sim[foresteps:]])
    else:
      k_sim_all = np.append(k_sim_all,np.array([k_sim[foresteps:]]))
      I_sim_all = np.append(I_sim_all,np.array([I_sim[foresteps:]]))
  return k_sim_all, I_sim_all

In [5]:
# Moment function
def data_moments(kvals,Ivals):
  if kvals.ndim == 1: 
    k_mean = kvals.mean()
    k_var = kvals.var()
  elif kvals.ndim == 2: 
    k_mean = kvals.mean(axis=1)
    k_var = kvals.var(axis=1)
  if Ivals.ndim == 1: 
    I_mean = Ivals.mean()
  elif Ivals.ndim == 2: 
    I_mean = Ivals.mean(axis=1)
  return np.array(k_mean), np.array(I_mean), np.array(k_var)

In [6]:
# Error function
def err_vec(data_kvals,data_Ivals,sim_kvals,sim_Ivals):

    k_mean_data, I_mean_data, k_var_data = data_moments(data_kvals,data_Ivals)
    moms_data = np.array([k_mean_data, k_mean_data, k_var_data])
    k_mean_sim, I_mean_sim, k_var_sim = data_moments(sim_kvals,sim_Ivals)
    k_mean_model = k_mean_sim.mean()
    I_mean_model = I_mean_sim.mean()
    k_var_model = k_var_sim.mean()
    moms_model = np.array([[k_mean_model], [I_mean_model], [k_var_model]])
    err_vec = (moms_model - moms_data) / moms_data
    
    return err_vec

In [7]:
# Criterion
def criterion(params, *args):
    
    theta, rho, sigma = params
    real_k, real_I, W_hat = args
    sim_kvals, sim_Ivals = linear_sim(params,times=10)
    err = err_vec(real_k,real_I,sim_kvals,sim_Ivals)
    crit_val = np.matmul(np.matmul(err.T, W_hat), err) 
    
    return crit_val

In [ ]:
# Simulate data with initial parameters

theta_low = 0.5
theta_high = 0.9
theta_npts = int((theta_high-theta_low)/0.1)+1
theta_grid = np.linspace(theta_low,theta_high,theta_npts)

rho_low = 0.5
rho_high = 0.9
rho_npts = int((rho_high-rho_low)/0.1)+1
rho_grid = np.linspace(rho_low,rho_high,rho_npts)

sigma_low = 0.0
sigma_high = 0.3
sigma_npts = int((sigma_high-sigma_low)/0.05)+1
sigma_grid = np.linspace(sigma_low,sigma_high,sigma_npts)

theta_real_low = 0.6
theta_real_high = 0.8
theta_real_npts = int((theta_real_high-theta_real_low)/0.05)+1
theta_real_grid = np.linspace(theta_real_low,theta_real_high,theta_real_npts)

rho_real_low = 0.6
rho_real_high = 0.8
rho_real_npts = int((rho_real_high-rho_real_low)/0.05)+1
rho_real_grid = np.linspace(rho_real_low,rho_real_high,rho_real_npts)

sigma_real_low = 0.05
sigma_real_high = 0.25
sigma_real_npts = int((sigma_real_high-sigma_real_low)/0.05)+1
sigma_real_grid = np.linspace(sigma_real_low,sigma_real_high,sigma_real_npts)

W_hat_0 = np.eye(3)
#arg_0 = (real_k, real_I, W_hat_0)

'''
Q_min = 10E+98
for i in tqdm(range(theta_npts)):
  for j in tqdm(range(rho_npts)):
    for k in tqdm(range(sigma_npts)):
      params_grid = [theta_grid[i],rho_grid[j],sigma_grid[k]]
      Q = criterion(params_grid,real_k, real_I, W_hat_0)
      if Q < Q_min:
        Q_min = Q
        opt_params = params_grid
      print([i,j,k])
print(opt_params)
'''

def smm_calc(i,j,k):
  params_grid = [theta_grid[i],rho_grid[j],sigma_grid[k]]
  Q = criterion(params_grid,real_k,real_I,W_hat_0)
  return params_grid, Q

real_grid = []
for l in range(theta_real_npts): 
  for m in range(rho_real_npts): 
    for n in range(sigma_real_npts): 
      real_grid.append((l,m,n))

grid = []
for i in range(theta_npts): 
  for j in range(rho_npts): 
    for k in range(sigma_npts): 
      grid.append((i,j,k))

colnames = ['grid', 'Q']

for l,m,n in tqdm(real_grid):
  # Real data
  para_real = [theta_real_grid[l],rho_real_grid[m],sigma_real_grid[n]]
  real_k, real_I = linear_sim(para_real)
  # optimize 
  res = Parallel(n_jobs=4)(delayed(smm_calc)(i,j,k) for i,j,k in tqdm(grid))
  x = pd.DataFrame(columns=colnames,data=res)
  filename = "result_(%g,%g,%g).csv" % (para_real[0],para_real[1],para_real[2])
  x.to_csv(filename)
  #x.to_csv("D://UChicago//Research//Python//x.csv")
